In [24]:
#libraries
import requests
import gzip
from io import BytesIO
import os

import numpy as np
import pandas as pd
from pandas_dq import dq_report

from sklearn.linear_model import LinearRegression

import folium
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# ETL

Télécharger et décompresser les fichiers de données nécessaires à créer notre dataset
- Il s'agit de données météos mensuelles de diverses stations "météos france", entre 1950 et 2022, par département

In [ ]:
if False:
    
# Répertoire de destination pour les fichiers décompressés
output_dir = "decompressed_files"
os.makedirs(output_dir, exist_ok=True)

# Générer les URLs pour les fichiers de 01 à 95
base_url = "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/MENS/MENSQ_{:02d}_previous-1950-2022.csv.gz"
urls = [base_url.format(i) for i in range(1, 96)]

def download_and_extract_csv(url, output_dir):
    # Télécharger le fichier
    response = requests.get(url)
    response.raise_for_status()  # Vérifier que la requête a réussi

    # Nom du fichier de sortie
    file_name = os.path.basename(url).replace(".gz", "")
    output_path = os.path.join(output_dir, file_name)

    # Décompresser le fichier et l'enregistrer
    with gzip.open(BytesIO(response.content), 'rt') as f_in:
        with open(output_path, 'w') as f_out:
            f_out.write(f_in.read())

    print(f"Fichier décompressé et enregistré : {output_path}")

# Télécharger et décompresser chaque fichier
for url in urls:
    download_and_extract_csv(url, output_dir)

Création du dataset à l'échelle "FRANCE"
- relier tous les fichiers de decompressed_files dans un seul fichier csv "France"

In [30]:
# Lister les fichiers à lire
files = [os.path.join("decompressed_files", file) for file in os.listdir("decompressed_files")]

# Lire les fichiers et les concaténer
dfs = [pd.read_csv(file, sep=";") for file in files]
df_france = pd.concat(dfs)

# Enregistrer le fichier final
output_path = "France.csv"
df_france.to_csv(output_path, index=False)
print(f"Fichier final enregistré : {output_path}")

Fichier final enregistré : France.csv



- descriptif des données sur mensq_descriptif_champs_323-Original.csv
- la colonne FFM sera notre target (vitesse du vent)

In [52]:
df_france = pd.read_csv("France.csv")

In [53]:
df_france.shape

(3178005, 162)

In [54]:
df_france.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,RR,QRR,NBRR,RR_ME,RRAB,QRRAB,RRABDAT,NBJRR1,NBJRR5,NBJRR10,NBJRR30,NBJRR50,NBJRR100,PMERM,QPMERM,NBPMERM,PMERMINAB,QPMERMINAB,PMERMINABDAT,TX,QTX,NBTX,TX_ME,TXAB,QTXAB,TXDAT,TXMIN,QTXMIN,TXMINDAT,NBJTX0,NBJTX25,NBJTX30,NBJTX35,NBJTXI20,NBJTXI27,NBJTXS32,TN,QTN,NBTN,TN_ME,TNAB,QTNAB,TNDAT,TNMAX,QTNMAX,TNMAXDAT,NBJTN5,NBJTN10,NBJTNI10,NBJTNI15,NBJTNI20,NBJTNS20,NBJTNS25,NBJGELEE,TAMPLIM,QTAMPLIM,TAMPLIAB,QTAMPLIAB,TAMPLIABDAT,NBTAMPLI,TM,QTM,NBTM,TMM,QTMM,NBTMM,NBJTMS24,TMMIN,QTMMIN,TMMINDAT,TMMAX,QTMMAX,TMMAXDAT,UNAB,QUNAB,UNABDAT,NBUN,UXAB,QUXAB,UXABDAT,NBUX,UMM,QUMM,NBUM,TSVM,QTSVM,NBTSVM,ETP,QETP,FXIAB,QFXIAB,DXIAB,QDXIAB,FXIDAT,NBJFF10,NBJFF16,NBJFF28,NBFXI,FXI3SAB,QFXI3SAB,DXI3SAB,QDXI3SAB,FXI3SDAT,NBJFXI3S10,NBJFXI3S16,NBJFXI3S28,NBFXI3S,FXYAB,QFXYAB,DXYAB,QDXYAB,FXYABDAT,NBJFXY8,NBJFXY10,NBJFXY15,NBFXY,FFM,QFFM,NBFFM,INST,QINST,NBINST,NBSIGMA0,NBSIGMA20,NBSIGMA80,GLOT,QGLOT,NBGLOT,DIFT,QDIFT,NBDIFT,DIRT,QDIRT,NBDIRT,HNEIGEFTOT,QHNEIGEFTOT,HNEIGEFAB,QHNEIGEFAB,HNEIGEFDAT,NBHNEIGEF,NBJNEIG,NBJHNEIGEF1,NBJHNEIGEF5,NBJHNEIGEF10,NBJSOLNG,NEIGETOTM,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBJGREL,NBJORAG,NBJBROU
0,1010001,ANGLEFORT,45.9,5.766667,280,195001,49.5,1.0,31.0,NaN,18.0,1.0,16.0,5.0,4.0,3.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1010001,ANGLEFORT,45.9,5.766667,280,195002,237.6,1.0,28.0,NaN,51.0,1.0,9.0,12.0,12.0,8.0,3.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1010001,ANGLEFORT,45.9,5.766667,280,195003,23.8,1.0,31.0,NaN,8.5,1.0,16.0,7.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010001,ANGLEFORT,45.9,5.766667,280,195005,83.6,1.0,31.0,NaN,12.8,1.0,15.0,11.0,8.0,4.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010001,ANGLEFORT,45.9,5.766667,280,195006,75.3,1.0,30.0,NaN,36.7,1.0,15.0,6.0,4.0,2.

In [55]:
#colonne FFM vide (pas de données sur la vitesse du vent)
df_france["FFM"].isnull().sum()

2837357

Beaucoup de data mais également beaucoup de lignes sans vitesse de vent. Les supprimer nous permettra quand même d'en garder +340000.

3178005 - 2837357 =    340 648 lignes exploitables depuis 1950
2468772 - 2159584 =    308 188 lignes exploitables depuis 1970
2013101 - 1721876 =    291 225 lignes exploitables depuis 1980
1537421 - 1270297 =    267 124 lignes exploitables depuis 1990 

Après exploration voici les décisions :
- suppressions des lignes sans données "FFM"
- mais ne pas travailler sur une temporalité plus courte (1980 à 2022 par exemple) et garder le dataset entre 1950 et 2022
- supprimer un certain nombre de colonnes qui ne nous semble pas du tout utile
- mais en garder un certain nombre même si nous ne les exploiterons pas forcément
- les renommer avec des noms plus parlant (voir mensq_descriptif_champs_323-Clean.csv)
- rajouter une colonne département et région pour nos tests modèles
- convertir la colonne AAAAMM en datetime et créer une colonne year et month

In [56]:
# Supprimer les lignes avec des valeurs manquantes dans la colonne "FMM"
df_france = df_france.dropna(subset=["FFM"])

In [57]:
df_france.shape

(340648, 162)

In [58]:
# Sélectionner les colonnes à conserver
columns_to_keep = [
    "NUM_POSTE", "NOM_USUEL", "LAT", "LON", "ALTI", "AAAAMM", "RR", "PMERM", "TX", "TXAB", "TXMIN",
    "NBJGELEE", "UNAB", "UXAB", "UMM", "FFM", "FXIAB", "DXIAB", "NBJFF10", "FXI3SAB", "DXI3SAB",
    "NBJFXI3S10", "FXYAB", "DXYAB", "NBJFXY10"
]
df_france = df_france[columns_to_keep]

# Enregistrer le fichier final
output_path = "France_clean.csv"
df_france.to_csv(output_path, index=False)
print(f"Fichier final enregistré : {output_path}")

Fichier final enregistré : France_clean.csv


In [59]:
df_france.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,RR,PMERM,TX,TXAB,TXMIN,NBJGELEE,UNAB,UXAB,UMM,FFM,FXIAB,DXIAB,NBJFF10,FXI3SAB,DXI3SAB,NBJFXI3S10,FXYAB,DXYAB,NBJFXY10
632,1014002,ARBENT,46.278167,5.669,534,200403,23.8,NaN,12.7,24.7,3.2,4.0,18.0,99.0,78.0,2.1,14.0,300.0,6.0,NaN,NaN,NaN,7.0,360.0,0.0
633,1014002,ARBENT,46.278167,5.669,534,200404,44.8,NaN,14.5,22.9,5.5,10.0,24.0,100.0,76.0,2.1,15.0,220.0,17.0,NaN,NaN,NaN,9.0,210.0,0.0
634,1014002,ARBENT,46.278167,5.669,534,200405,95.0,NaN,18.8,27.0,7.3,3.0,24.0,100.0,74.0,1.9,14.0,340.0,13.0,NaN,NaN,NaN,9.0,330.0,0.0
635,1014002,ARBENT,46.278167,5.669,534,200406,56.3,NaN,23.6,31.4,13.3,0.0,17.0,99.0,70.0,2.1,18.0,160.0,15.0,NaN,NaN,NaN,8.0,170.0,0.0
636,1014002,ARBENT,46.278167,5.669,534,200407,61.3,NaN,25.3,33.3,17.0,0.0,24.0,99.0,68.0,2.0,18.0,210.0,14.0,NaN,NaN,NaN,10.0,150.0,1.0


In [60]:
df_france.shape

(340648, 25)

In [61]:
# renomer les colonnes
df_france = df_france.rename(columns={
    "RR": "precip_cumul_mensu",
    "TX": "temp_moy_mensu_max",
    "TXAB": "temp_max_mensu",
    "TXMIN": "temp_min_mensu",
    "UNAB": "humi_min_mensu",
    "UXAB": "humi_max_mensu",
    "UMM": "humi_moy_mensu",
    "FFM": "vent_speed_inst_moy_mensu",
    "FXIAB": "vent_speed_inst_max_mensu",
    "DXIAB": "vent_dir_inst",
    "NBJFF10": "vent_nbjour_inst_speedsup10ms",
    "FXYAB": "vent_speed_10mn_max_mensu",
    "DXYAB": "vent_dir_10mn",
    "NBJFXY10": "vent_nbjour_10mn_speedsup10ms"
})

In [62]:
# Modifier la colonne AAAAMM en datetime
df_france["AAAAMM"] = pd.to_datetime(df_france["AAAAMM"], format="%Y%m")

In [63]:
# Créer une colonne pour l'année et le mois et les ajouter au début du DataFrame
df_france["Year"] = df_france["AAAAMM"].dt.year
df_france["Month"] = df_france["AAAAMM"].dt.month

# et les ajouter au début du DataFrame
cols = list(df_france.columns)
cols.insert(cols.index("AAAAMM") + 1, cols.pop(cols.index("Year")))
cols.insert(cols.index("Year") + 1, cols.pop(cols.index("Month")))
df_france = df_france[cols]

In [64]:
# Convertir les colonnes de vitesse du vent de m/s en km/h
df_france["vent_speed_inst_moy_mensu"] = df_france["vent_speed_inst_moy_mensu"] * 3.6
df_france["vent_speed_inst_max_mensu"] = df_france["vent_speed_inst_max_mensu"] * 3.6
df_france["vent_speed_10mn_max_mensu"] = df_france["vent_speed_10mn_max_mensu"] * 3.6
df_france = df_france.round({"vent_speed_inst_moy_mensu": 1, "vent_speed_inst_max_mensu": 1, "vent_speed_10mn_max_mensu": 1})

In [65]:
# créer les colonnes département et région
df_france["departement_num"] = df_france["NUM_POSTE"].astype(str).str[:2]
df_france.shape

(340648, 28)

In [66]:
departement_mapping = {
    "01": ("Ain", "AUVERGNE RHONE ALPES"),
    "02": ("Aisne", "HAUTS DE FRANCE"),
    "03": ("Allier", "AUVERGNE RHONE ALPES"),
    "04": ("Alpes-de-Haute-Provence", "PROVENCE ALPES COTE D'AZUR"),
    "05": ("Hautes-Alpes", "PROVENCE ALPES COTE D'AZUR"),
    "06": ("Alpes-Maritimes", "PROVENCE ALPES COTE D'AZUR"),
    "07": ("Ardeche", "AUVERGNE RHONE ALPES"),
    "08": ("Ardennes", "GRAND EST"),
    "09": ("Ariege", "OCCITANIE"),
    "10": ("Aube", "GRAND EST"),
    "11": ("Aude", "OCCITANIE"),
    "12": ("Aveyron", "OCCITANIE"),
    "13": ("Bouches-du-Rhone", "PROVENCE ALPES COTE D'AZUR"),
    "14": ("Calvados", "NORMANDIE"),
    "15": ("Cantal", "AUVERGNE RHONE ALPES"),
    "16": ("Charente", "NOUVELLE AQUITAINE"),
    "17": ("Charente-Maritime", "NOUVELLE AQUITAINE"),
    "18": ("Cher", "CENTRE VAL DE LOIRE"),
    "19": ("Correze", "NOUVELLE AQUITAINE"),
    "2A": ("Corse-du-Sud", "CORSE"),
    "2B": ("Haute-CORSE", "CORSE"),
    "21": ("Cote-d'Or", "BOURGOGNE FRANCHE COMTE"),
    "22": ("Cotes d'Armor", "BRETAGNE"),
    "23": ("Creuse", "NOUVELLE AQUITAINE"),
    "24": ("Dordogne", "NOUVELLE AQUITAINE"),
    "25": ("Doubs", "BOURGOGNE FRANCHE COMTE"),
    "26": ("Drome", "AUVERGNE RHONE ALPES"),
    "27": ("Eure", "NORMANDIE"),
    "28": ("Eure-et-Loir", "CENTRE VAL DE LOIRE"),
    "29": ("Finistere", "BRETAGNE"),
    "30": ("Gard", "OCCITANIE"),
    "31": ("Haute-Garonne", "OCCITANIE"),
    "32": ("Gers", "OCCITANIE"),
    "33": ("Gironde", "NOUVELLE AQUITAINE"),
    "34": ("Herault", "OCCITANIE"),
    "35": ("Ille-et-Vilaine", "BRETAGNE"),
    "36": ("Indre", "CENTRE VAL DE LOIRE"),
    "37": ("Indre-et-Loire", "CENTRE VAL DE LOIRE"),
    "38": ("Isere", "AUVERGNE RHONE ALPES"),
    "39": ("Jura", "BOURGOGNE FRANCHE COMTE"),
    "40": ("Landes", "NOUVELLE AQUITAINE"),
    "41": ("Loir-et-Cher", "CENTRE VAL DE LOIRE"),
    "42": ("Loire", "AUVERGNE RHONE ALPES"),
    "43": ("Haute-Loire", "AUVERGNE RHONE ALPES"),
    "44": ("Loire-Atlantique", "PAYS DE LA LOIRE"),
    "45": ("Loiret", "CENTRE VAL DE LOIRE"),
    "46": ("Lot", "OCCITANIE"),
    "47": ("Lot-et-Garonne", "NOUVELLE AQUITAINE"),
    "48": ("Lozere", "OCCITANIE"),
    "49": ("Maine-et-Loire", "PAYS DE LA LOIRE"),
    "50": ("Manche", "NORMANDIE"),
    "51": ("Marne", "GRAND EST"),
    "52": ("Haute-Marne", "GRAND EST"),
    "53": ("Mayenne", "PAYS DE LA LOIRE"),
    "54": ("Meurthe-et-Moselle", "GRAND EST"),
    "55": ("Meuse", "GRAND EST"),
    "56": ("Morbihan", "BRETAGNE"),
    "57": ("Moselle", "GRAND EST"),
    "58": ("Nievre", "BOURGOGNE FRANCHE COMTE"),
    "59": ("Nord", "HAUTS DE FRANCE"),
    "60": ("Oise", "HAUTS DE FRANCE"),
    "61": ("Orne", "NORMANDIE"),
    "62": ("Pas-de-Calais", "HAUTS DE FRANCE"),
    "63": ("Puy-de-Dome", "AUVERGNE RHONE ALPES"),
    "64": ("Pyrenees-Atlantiques", "NOUVELLE AQUITAINE"),
    "65": ("Hautes-Pyrenees", "OCCITANIE"),
    "66": ("Pyrenees-Orientales", "OCCITANIE"),
    "67": ("Bas-Rhin", "GRAND EST"),
    "68": ("Haut-Rhin", "GRAND EST"),
    "69": ("Rhone", "AUVERGNE RHONE ALPES"),
    "70": ("Haute-Saone", "BOURGOGNE FRANCHE COMTE"),
    "71": ("Saone-et-Loire", "BOURGOGNE FRANCHE COMTE"),
    "72": ("Sarthe", "PAYS DE LA LOIRE"),
    "73": ("Savoie", "AUVERGNE RHONE ALPES"),
    "74": ("Haute-Savoie", "AUVERGNE RHONE ALPES"),
    "75": ("Paris", "ILE DE FRANCE"),
    "76": ("Seine-Maritime", "NORMANDIE"),
    "77": ("Seine-et-Marne", "ILE DE FRANCE"),
    "78": ("Yvelines", "ILE DE FRANCE"),
    "79": ("Deux-Sevres", "NOUVELLE AQUITAINE"),
    "80": ("Somme", "HAUTS DE FRANCE"),
    "81": ("Tarn", "OCCITANIE"),
    "82": ("Tarn-et-Garonne", "OCCITANIE"),
    "83": ("Var", "PROVENCE ALPES COTE D'AZUR"),
    "84": ("Vaucluse", "PROVENCE ALPES COTE D'AZUR"),
    "85": ("Vendee", "PAYS DE LA LOIRE"),
    "86": ("Vienne", "NOUVELLE AQUITAINE"),
    "87": ("Haute-Vienne", "NOUVELLE AQUITAINE"),
    "88": ("Vosges", "GRAND EST"),
    "89": ("Yonne", "BOURGOGNE FRANCHE COMTE"),
    "90": ("Territoire de Belfort", "BOURGOGNE FRANCHE COMTE"),
    "91": ("Essonne", "ILE DE FRANCE"),
    "92": ("Hauts-de-Seine", "ILE DE FRANCE"),
    "93": ("Seine-St-Denis", "ILE DE FRANCE"),
    "94": ("Val-de-Marne", "ILE DE FRANCE"),
    "95": ("Val-d'Oise", "ILE DE FRANCE"),
    "971": ("Guadeloupe", "GUADELOUPE"),
    "972": ("Martinique", "MARTINIQUE"),
    "973": ("Guyane", "GUYANE"),
    "974": ("La Reunion", "LA REUNION"),
    "976": ("Mayotte", "MAYOTTE")
}

In [67]:
df_france["departement_name"] = df_france["departement_num"].map(lambda x: departement_mapping.get(x, (None, None))[0])
df_france["region"] = df_france["departement_num"].map(lambda x: departement_mapping.get(x, (None, None))[1])
df_france.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,PMERM,temp_moy_mensu_max,temp_max_mensu,temp_min_mensu,NBJGELEE,humi_min_mensu,humi_max_mensu,humi_moy_mensu,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,FXI3SAB,DXI3SAB,NBJFXI3S10,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
632,1014002,ARBENT,46.278167,5.669,534,2004-03-01,2004,3,23.8,NaN,12.7,24.7,3.2,4.0,18.0,99.0,78.0,7.6,50.4,300.0,6.0,NaN,NaN,NaN,25.2,360.0,0.0,10,Aube,GRAND EST
633,1014002,ARBENT,46.278167,5.669,534,2004-04-01,2004,4,44.8,NaN,14.5,22.9,5.5,10.0,24.0,100.0,76.0,7.6,54.0,220.0,17.0,NaN,NaN,NaN,32.4,210.0,0.0,10,Aube,GRAND EST
634,1014002,ARBENT,46.278167,5.669,534,2004-05-01,2004,5,95.0,NaN,18.8,27.0,7.3,3.0,24.0,100.0,74.0,6.8,50.4,340.0,13.0,NaN,NaN,NaN,32.4,330.0,0.0,10,Aube,GRAND EST
635,1014002,ARBENT,46.278167,5.669,534,2004-06-01,2004,6,56.3,NaN,23.6,31.4,13.3,0.0,17.0,99.0,70.0,7.6,64.8,160.0,15.0,NaN,NaN,NaN,28.8,170.0,0.0,10,Aube,GRAND EST
636,1014002,ARBENT,46.278167,5.669,534,2004-07-01,2004,7,61.3,NaN,25.3,33.3,17.0,0.0,24.0,99.0,68.0,7.2,64.8,210.0,14.0,NaN,NaN,NaN,36.0,150.0,1.0,10,Aube,GRAND EST


In [68]:
# Enregistrer le fichier
output_path = "datameteo_france_1950-2022_clean.csv"
df_france.to_csv(output_path, index=False)
print(f"Fichier final enregistré : {output_path}")

Fichier final enregistré : datameteo_france_1950-2022_clean.csv


In [70]:
# vérification des données manquantes :
df_france = pd.read_csv("datameteo_france_1950-2022_clean.csv")
dq_report(df_france, verbose=1)

C:\Users\dsgat\AppData\Local\Temp\ipykernel_34084\896320115.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dq_report(df_france, verbose=1)
C:\Users\dsgat\AppData\Local\Temp\ipykernel_34084\896320115.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dq_report(df_france, verbose=1)


    All variables classified into correct types.


C:\Users\dsgat\AppData\Local\Temp\ipykernel_34084\896320115.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dq_report(df_france, verbose=1)


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
NUM_POSTE,int64,0.000000,0,1014002.000000,95690001.000000,No issue
NOM_USUEL,object,0.000000,0,,,No issue
LAT,float64,0.000000,NA,41.374833,51.055833,No issue
LON,float64,0.000000,NA,-16.000000,9.541500,Column has 261 outliers greater than upper bound (13.00) or lower than lower bound(-6.85). Cap them or remove them.
ALTI,int64,0.000000,0,1.000000,3845.000000,Column has 36128 outliers greater than upper bound (852.00) or lower than lower bound(-396.00). Cap them or remove them.
AAAAMM,object,0.000000,0,,,Possible high cardinality column with 876 unique values: Use hash encoding or text embedding to reduce dimension.
Year,int64,0.000000,0,1950.000000,2022.000000,Possible date-time colum: transform before modeling step.
Month,int64,0.000000,0,1.000000,12.000000,No issue
precip_cumul_mensu,float64,3.791891,NA,0.000000,1075.000000,"12917 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 12058 outliers greater than upper bound (182.40) or lower than lower bound(-56.00). Cap them or remove them."
PMERM,float64,61.739097,NA,855.900000,1039.600000,"210313 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 6284 outliers greater than upper bound (1026.00) or lower than lower bound(1007.60). Cap them or remove them."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
NUM_POSTE,int64,0.000000,0,1014002.0,95690001.0,No issue
NOM_USUEL,object,0.000000,0,,,No issue
LAT,float64,0.000000,NA,41.374833,51.055833,No issue
LON,float64,0.000000,NA,-16.0,9.5415,Column has 261 outliers greater than upper bound (13.00) or lower than lower bound(-6.85). Cap them or remove them.
ALTI,int64,0.000000,0,1.0,3845.0,Column has 36128 outliers greater than upper bound (852.00) or lower than lower bound(-396.00). Cap them or remove them.
AAAAMM,object,0.000000,0,,,Possible high cardinality column with 876 unique values: Use hash encoding or text embedding to reduce dimension.
Year,int64,0.000000,0,1950.0,2022.0,Possible date-time colum: transform before modeling step.
Month,int64,0.000000,0,1.0,12.0,No issue
precip_cumul_mensu,float64,3.791891,NA,0.0,1075.0,"12917 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 12058 outliers greater than upper bound (182.40) or lower than lower bound(-56.00). Cap them or remove them."
PMERM,float64,61.739097,NA,855.9,1039.6,"210313 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 6284 outliers greater than upper bound (1026.00) or lower than lower bound(1007.60). Cap them or remove them."


Aucune valeurs manquante pour le vent ! Des colonnes avec des valeurs manquantes plus qu'importantes à supprimer. Revoir également les noms de département et région

In [71]:
# Supprimer les colonnes
columns_to_drop = ["PMERM", "FXI3SAB", "DXI3SAB", "NBJFXI3S10"]
df_france = df_france.drop(columns=columns_to_drop)
df_france.shape

(340648, 26)

In [75]:
#valeurs manquantes avec département manquant
df_france[df_france["departement_name"].isnull()]

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_moy_mensu_max,temp_max_mensu,temp_min_mensu,NBJGELEE,humi_min_mensu,humi_max_mensu,humi_moy_mensu,vent_speed_inst_moy_mensu,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region
2534,2037002,AULNOIS-SS-LAON,49.595667,3.610333,78,1989-01-01,1989,1,31.8,6.3,11.5,1.4,10.0,71.0,100.0,98.0,10.8,68.4,180.0,7.0,NaN,NaN,NaN,20,NaN,NaN
2535,2037002,AULNOIS-SS-LAON,49.595667,3.610333,78,1989-02-01,1989,2,46.4,7.8,14.7,-0.9,9.0,66.0,100.0,94.0,15.8,79.2,280.0,15.0,NaN,NaN,NaN,20,NaN,NaN
2536,2037002,AULNOIS-SS-LAON,49.595667,3.610333,78,1989-03-01,1989,3,71.8,13.7,22.4,7.6,1.0,37.0,100.0,84.0,16.6,82.8,300.0,21.0,NaN,NaN,NaN,20,NaN,NaN
2537,2037002,AULNOIS-SS-LAON,49.595667,3.610333,78,1989-04-01,1989,4,117.0,11.8,19.7,5.9,3.0,29.0,100.0,90.0,13.0,68.4,240.0,17.0,NaN,NaN,NaN,20,NaN,NaN
2538,2037002,AULNOIS-SS-LAON,49.595667,3.610333,78,1989-05-01,1989,5,31.0,22.0,28.7,15.1,0.0,20.0,100.0,74.0,10.1,57.6,240.0,10.0,NaN,NaN,NaN,20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91754,20354005,VIVARIO-COL DE LA SERRA,42.169667,9.165833,807,2002-04-01,2002,4,96.1,16.1,22.0,10.0,0.0,26.0,99.0,NaN,8.6,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN
91755,20354005,VIVARIO-COL DE LA SERRA,42.169667,9.165833,807,2002-05-01,2002,5,45.3,20.0,25.0,14.0,0.0,30.0,98.0,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN
91756,20354005,VIVARIO-COL DE LA SERRA,42.169667,9.165833,807,2002-06-01,2002,6,41.2,27.1,34.0,18.0,0.0,27.0,98.0,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN
91757,20354005,VIVARIO-COL DE LA SERRA,42.169667,9.165833,807,2002-07-01,2002,7,24.0,27.6,32.0,20.0,0.0,21.0,98.0,NaN,7.2,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN


pb sur les valeurs 02 / 20 / 2A / 2B à voir avec Tristan

In [ ]:
# Enregistrer le fichier
output_path = "datameteo_france_1950-2022_clean_02.csv"
df_france.to_csv(output_path, index=False)
print(f"Fichier final enregistré : {output_path}")